In [14]:
import os, stat

In [15]:
import Waveforms
from Waveforms import *

In [16]:
ROOT_PATH = globals()['_dh'][0]
PLOT_PATH_ROOT = os.path.join(ROOT_PATH, f"plots/")
SLURM_PATH_ROOT = os.path.join(ROOT_PATH, "slurm_scripts/")

venv_activate_path = "/home/mcheung1/Ringdown/jaxqualin/bin/activate"
slurm_email = "hcheung5@jhu.edu"
python_script_path = os.path.join(ROOT_PATH, "convergence_test.py")

In [17]:
runname = "convergence_test_3"
SXS_num = "0305"
runname_full = f"{SXS_num}_{runname}"

opts_1 = "-l -mn 200000 -tl 0 -th 50 -tn 501 -oih -1 -xt 1e-11 -gt 1e-11 -ft 1e-11"
guess_num = 10
N_list = [5, 6, 7, 8, 9, 10]

In [18]:
h_dict, _ = get_relevant_lm_waveforms_SXS(SXS_num)

Found the following files to load from the SXS catalog:
    SXS:BBH:0305v5/Lev6/rhOverM_Asymptotic_GeometricUnits_CoM.h5
Found the following files to load from the SXS catalog:
    SXS:BBH:0305v5/Lev6/metadata.json


In [19]:
relevant_lm_list = [tuple(map(int,str.split('.'))) for str in list(h_dict.keys())]
lm_num = len(relevant_lm_list)

In [20]:
slurm_dir = os.path.join(SLURM_PATH_ROOT, f"convergence/")
os.makedirs(slurm_dir, exist_ok = True)

bash_dir = os.path.join(SLURM_PATH_ROOT, f"convergence/bash_scripts/{runname_full}/")
os.makedirs(bash_dir, exist_ok = True)

In [21]:
N_string = ""
for N_free in N_list:
    N_string += f"{N_free} "

mem = int(lm_num*2)

In [22]:
slurm_head = f'''#!/bin/bash
#SBATCH --job-name={runname_full}  
#SBATCH --mail-type=ALL          
#SBATCH --mail-user={slurm_email}             
#SBATCH --nodes=1
#SBATCH --ntasks-per-node={lm_num}
#SBATCH --cpus-per-task=1
#SBATCH --mem={mem}gb                     
#SBATCH --time=24:00:00               
#SBATCH --output={SLURM_PATH_ROOT}/slurm_out/convergence/{runname_full}_%j.log   
#SBATCH --partition=shared
#SBATCH --account=cit122
#SBATCH --array=0-{guess_num-1}

pwd; hostname; date

source {venv_activate_path}
export PYTHONPATH=$PYTHONPATH:/home/mcheung1/Ringdown/jaxqualin/Jaxqualin

'''


In [23]:
for lm in relevant_lm_list:
    l, m = lm
    lm_file_loc = os.path.join(bash_dir, f"lm_{l}{m}.sh")
    bash_script = f'''#!/bin/bash

source {venv_activate_path}
export PYTHONPATH=$PYTHONPATH:/home/mcheung1/Ringdown/jaxqualin/Jaxqualin

script="{python_script_path}"
runname="{SXS_num}"

opts="-n $SLURM_ARRAY_TASK_ID {opts_1}"

'''
    for N_free in N_list:
        line = f"python $script $runname {l} {m} {N_free} {guess_num} {runname} $opts \n"
        bash_script += line
    with open(lm_file_loc, 'w') as f:
        f.write(bash_script)
    os.chmod(lm_file_loc, stat.S_IRWXU)

In [24]:
print(bash_script)

#!/bin/bash

source /home/mcheung1/Ringdown/jaxqualin/bin/activate
export PYTHONPATH=$PYTHONPATH:/home/mcheung1/Ringdown/jaxqualin/Jaxqualin

script="/home/mcheung1/Ringdown/jaxqualin/Jaxqualin/convergence_test.py"
runname="0305"

opts="-n $SLURM_ARRAY_TASK_ID -l -mn 200000 -tl 0 -th 50 -tn 501 -oih -1 -xt 1e-11 -gt 1e-11 -ft 1e-11"

python $script $runname 6 6 5 10 convergence_test_3 $opts 
python $script $runname 6 6 6 10 convergence_test_3 $opts 
python $script $runname 6 6 7 10 convergence_test_3 $opts 
python $script $runname 6 6 8 10 convergence_test_3 $opts 
python $script $runname 6 6 9 10 convergence_test_3 $opts 
python $script $runname 6 6 10 10 convergence_test_3 $opts 



In [25]:
slurm_script = slurm_head

for lm in relevant_lm_list:
    l, m = lm
    lm_file_loc = os.path.join(bash_dir, f"lm_{l}{m}.sh")
    slurm_main_lm = ""
    N_string = ""
    for N_free in N_list:
        N_string += f"{N_free} "
    slurm_main_lm_N = (f"srun --exclusive -n 1 -c 1 {lm_file_loc} {l} {m} &\n")
    slurm_script += slurm_main_lm_N

slurm_script += '''
wait

date'''


In [26]:
print(slurm_script)

#!/bin/bash
#SBATCH --job-name=0305_convergence_test_3  
#SBATCH --mail-type=ALL          
#SBATCH --mail-user=hcheung5@jhu.edu             
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=1
#SBATCH --mem=16gb                     
#SBATCH --time=24:00:00               
#SBATCH --output=/home/mcheung1/Ringdown/jaxqualin/Jaxqualin/slurm_scripts//slurm_out/convergence/0305_convergence_test_3_%j.log   
#SBATCH --partition=shared
#SBATCH --account=cit122
#SBATCH --array=0-9

pwd; hostname; date

source /home/mcheung1/Ringdown/jaxqualin/bin/activate
export PYTHONPATH=$PYTHONPATH:/home/mcheung1/Ringdown/jaxqualin/Jaxqualin

srun --exclusive -n 1 -c 1 /home/mcheung1/Ringdown/jaxqualin/Jaxqualin/slurm_scripts/convergence/bash_scripts/0305_convergence_test_3/lm_22.sh 2 2 &
srun --exclusive -n 1 -c 1 /home/mcheung1/Ringdown/jaxqualin/Jaxqualin/slurm_scripts/convergence/bash_scripts/0305_convergence_test_3/lm_32.sh 3 2 &
srun --exclusive -n 1 -c 1 /home/mcheung1/Ringdown/jaxq

In [14]:
with open(os.path.join(slurm_dir, f"{runname_full}.sbatch"), 'w') as f:
    f.write(slurm_script)